In [12]:
from cyecca.symbolic import sympy_to_casadi, casadi_to_sympy
import sympy.physics.mechanics as me
import sympy

me.init_vprinting()
t = sympy.symbols('t')

# parameters
g, m, J_x, J_y, J_z, J_xz = sympy.symbols('g, m, J_x, J_y, J_z, J_xz')
CL_0, CL_a, CD_0, k_CDCL, x_ac = sympy.symbols('CL_0, CL_a, CD_0, k_CDCL, x_ac')
p_vect = sympy.Matrix([g, m, J_x, J_y, J_z, J_xz, J_x, J_y, J_z, J_xz, CL_0, CL_a, CD_0, k_CDCL])
p_dict = {
    g: 9.8,
    m: 0.01,
    J_x: 0.01,
    J_y: 0.01,
    J_z: 0.01,
    J_xz: 0.01,
    CL_0: 0,
    CL_a: 0.1,
    CD_0: 0,
    k_CDCL: 0.01,
    x_ac: 0.01
}

# states
p_n, p_e, h = me.dynamicsymbols('p_n, p_e, h')
U, V, W, P, Q, R = me.dynamicsymbols('U, V, W, P, Q, R')
phi, theta, psi = me.dynamicsymbols('phi, theta, psi')
x_vect = sympy.Matrix([U, V, W, P, Q, R, phi, theta, psi, p_n, p_e, h])

# inputs
thr, ail, el, rdr = me.dynamicsymbols('thr, ail, el, rdr')

# frames
V_T = sympy.sqrt(U*U + V*V + W*W)
alpha = sympy.atan(W/U)
beta = sympy.asin(V/V_T)
frame_e = me.ReferenceFrame('e')
frame_b = frame_e.orientnew('b', 'Body', (psi, theta, phi), rot_order='321')
omega_eb_b = P*frame_b.x + Q*frame_b.y + R*frame_b.z
kd_eqs_euler = (frame_b.ang_vel_in(frame_e) - omega_eb_b).to_matrix(frame_b)
frame_b.set_ang_vel(frame_e, omega_eb_b)
frame_s = frame_b.orientnew('s', 'Axis', (frame_b.y, -alpha))
frame_w = frame_s.orientnew('s', 'Axis', (frame_s.z, beta))

point_o = me.Point('o')
point_o.set_vel(frame_e, 0)

# center of mass of vehicle
V_eb_b = U*frame_b.x + V*frame_b.y + W*frame_b.z
point_cm = point_o.locatenew('o', p_n*frame_e.x + p_e*frame_e.y - h*frame_e.z)
point_cm.set_vel(frame_b, 0)
kd_eqs_nav = (V_eb_b - point_cm.vel(frame_e)).to_matrix(frame_b)
point_cm.set_vel(frame_e, V_eb_b)

# aerodynamic center
point_ac = point_cm.locatenew('ac', -x_ac*frame_b.x)
body = me.RigidBody('body', masscenter=point_cm, frame=frame_b, mass=m, inertia=(me.inertia(frame_b, J_x, J_y, J_z, 0, 0, J_xz), point_cm))

# deriviative of angular momentum
H_cm_e = body.angular_momentum(point_cm, frame_e)
H_dot_cm_e = H_cm_e.diff(t, frame_b) + frame_b.ang_vel_in(frame_e).cross(H_cm_e)

# derivative of linear momentum
L_e = body.linear_momentum(frame_e)
L_dot_e = L_e.diff(t, frame_b) + frame_b.ang_vel_in(frame_e).cross(L_e)

km = me.KanesMethod(
    frame=frame_e,
    q_ind=[p_n, p_e, h, phi, theta, psi],
    u_ind=[U, V, W, P, Q, R],
    kd_eqs=list(kd_eqs_euler) + list(kd_eqs_nav),
    bodies=[body],
    forcelist=[
        (point_cm, m*g*frame_e.z),
        (point_ac, 1*frame_w.z -1 * frame_w.x)
    ]
)
km.kanes_equations()

⎛⎡                       __________________                      ⎤             ↪
⎜⎢                      ╱           2                            ⎥             ↪
⎜⎢                     ╱           V                             ⎥             ↪
⎜⎢                    ╱   1 - ────────────                       ⎥             ↪
⎜⎢                   ╱         2    2    2                       ⎥             ↪
⎜⎢                 ╲╱         U  + V  + W            W           ⎥             ↪
⎜⎢   -g⋅m⋅sin(θ) - ──────────────────────── - ────────────────   ⎥             ↪
⎜⎢                            ________              ________     ⎥             ↪
⎜⎢                           ╱      2              ╱      2      ⎥             ↪
⎜⎢                          ╱      W              ╱      W       ⎥             ↪
⎜⎢                         ╱   1 + ──            ╱   1 + ── ⋅U   ⎥             ↪
⎜⎢                        ╱         2           ╱         2      ⎥             ↪
⎜⎢                      ╲╱  

In [21]:
mass_matrix = km.mass_matrix_full
forcing = km.forcing_full
mass_matrix

⎡1  0  0  0  0  0  0  0  0   0     0    0  ⎤
⎢                                          ⎥
⎢0  1  0  0  0  0  0  0  0   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  1  0  0  0  0  0  0   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  0  1  0  0  0  0  0   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  0  0  1  0  0  0  0   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  0  0  0  1  0  0  0   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  0  0  0  0  m  0  0   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  0  0  0  0  0  m  0   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  0  0  0  0  0  0  m   0     0    0  ⎥
⎢                                          ⎥
⎢0  0  0  0  0  0  0  0  0   Jₓ    0   J_xz⎥
⎢                                          ⎥
⎢0  0  0  0  0  0  0  0  0   0    J_y   0  ⎥
⎢                                          ⎥
⎣0  0  0  

In [22]:
forcing.simplify()

In [20]:
forcing

⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢⎛⎛                                                                            ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜─────────────────────────────────────────────────────────────────────────── ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜⎛  (-sin(φ)⋅sin(θ)⋅cos(ψ) + sin(ψ)⋅cos(φ))⋅sin(ψ)                           ↪
⎢⎜⎜⎜- ────────────────────────────────────────────── - sin(φ)⋅sin(ψ)⋅sin(θ) -  ↪
⎢⎜⎜⎝                      co

In [11]:
subs_no_dynamic = {
    Q: sympy.symbols('Q'),
    R: sympy.symbols('R')
}
rhs.subs(Q, subs_no_dynamic)

/home/jgoppert/cognipilot/tools/src/cyecca/.venv/lib/python3.12/site-packages/sympy/core/basic.py:1214: SymPyDeprecationWarning: 

Using non-Expr arguments in Mul is deprecated (in this case, one of
the arguments has type 'Dict').

If you really did intend to use a multiplication or addition operation with
this object, use the * or + operator instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#non-expr-args-deprecated
for details.

This has been deprecated since SymPy version 1.7. It
will be removed in a future version of SymPy.

  rv = self.func(*args)
/home/jgoppert/cognipilot/tools/src/cyecca/.venv/lib/python3.12/site-packages/sympy/printing/pretty/pretty.py:1933: SymPyDeprecationWarning: 

Using non-Expr arguments in Mul is deprecated (in this case, one of
the arguments has type 'Dict').

If you really did intend to use a multiplication or addition operation with
this object, use the * or + operator instead.

See https://docs.sympy.org/latest/explanatio

⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢⎛⎛                                                                            ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜─────────────────────────────────────────────────────────────────────────── ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜                                                                            ↪
⎢⎜⎜⎛  (-sin(φ)⋅sin(θ)⋅cos(ψ) + sin(ψ)⋅cos(φ))⋅sin(ψ)                           ↪
⎢⎜⎜⎜- ────────────────────────────────────────────── - sin(φ)⋅sin(ψ)⋅sin(θ) -  ↪
⎢⎜⎜⎝                      co

In [8]:
rhs = km.rhs()
sympy_to_casadi(rhs, cse=True)

unhandled type Q Q(t)
unhandled type R R(t)
unhandled type P P(t)
unhandled type U U(t)
unhandled type V V(t)
unhandled type psi psi(t)
unhandled type phi phi(t)
unhandled type theta theta(t)
unhandled type W W(t)


NotImplementedError: Wrong number or type of arguments for overloaded function 'substitute'.
  Possible prototypes are:
    substitute(DM,DM,DM)
    substitute([DM],[DM],[DM])
    substitute(SX,SX,SX)
    substitute([SX],[SX],[SX])
    substitute(MX,MX,MX)
    substitute([MX],[MX],[MX])
  You have: '(SX,SX,NoneType)'


In [173]:
point_cm.vel(frame_e)